In [1]:
!pip install -q -U torch transformers accelerate datasets
!pip install -q flash-attn

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


# Load tokenizer and model
model_name = "tchen175/llama3.1-8b-newsmtsc"
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16, 
    device_map="cuda",
    attn_implementation="flash_attention_2"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


2024-12-11 09:02:00.055230: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-11 09:02:00.840460: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
alpaca_prompt = """Determine the sentiment of the following article as -1 (negative), 0 (neutral), or 1 (positive).

### Article:
{article}

### Semantic label:
"""

## Eval our finetuned model


In [30]:
def predict_sentiment_logits(model, tokenizer, dataset, device='cuda'):
    
    model.to(device)
    model.eval()
    predictions = []
    
    for text in dataset['sentence']:
        prompt = alpaca_prompt.format(article = text)

        # Prepare input
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

        possible_outputs = ['-1', '0', '1']

        output_token_ids = [tokenizer(output, add_special_tokens=False).input_ids[0] for output in possible_outputs]
        
        # Get logits
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
        
        last_token_logits = logits[:, -1, :]  # Shape: (batch_size, vocab_size)

        # Filter logits for the possible outputs
        filtered_logits = last_token_logits[:, output_token_ids]  # Shape: (batch_size, len(possible_outputs))

        # Print filtered logits (comment this line for performance)
        # print("Logits for possible outputs:", filtered_logits)

        # Optionally, convert logits to probabilities
        probabilities = torch.softmax(filtered_logits, dim=-1)
        # print("Probabilities for possible outputs:", probabilities)  # Commented to avoid excessive I/O

        # Get the label with the highest probability
        predicted_label_index = torch.argmax(probabilities, dim=-1).item()
        predicted_label = possible_outputs[predicted_label_index]
        predictions.append(int(predicted_label))
    
    return predictions


### eval GPT4o

In [2]:
!pip install -q -U openai python-dotenv

In [1]:
import os
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

openai_client = OpenAI(
    base_url="https://api.openai.com/v1",
    api_key=api_key#enter api key
)

API Key:sk-proj-yKHe10Y7zEhFX1sncOFCT3BlbkFJh31ATScEQmYjQRlNpiLg


In [43]:
function_schema = {
        "name": "sentiment_analysis",
        "description": "Analyze the sentiment of the given article and return -1 (negative), 0 (neutral), or 1 (positive).",
        "parameters": {
            "type": "object",
            "properties": {
                "article": {
                    "type": "string",
                    "description": "The article text to analyze for sentiment."
                }
            },
            "type": "object",
            "properties": {
                "sentiment": {
                    "type": "integer",
                    "description": "The sentiment of the article: -1 for negative, 0 for neutral, 1 for positive."
                }
            },
            "required": ["article", 'sentiment']
        },
        
    }

In [44]:
import json

def predict_sentiment_with_function_calling(openai_client, dataset):
    # Define the function schema
    

    predictions = []

    for text in dataset['sentence']:
        # Call the OpenAI API with the function schema
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a sentiment analysis assistant."},
                {"role": "user", "content": f"Analyze the sentiment of this article: {text}"}
            ],
            functions=[function_schema],
            function_call={"name": "sentiment_analysis"}
        )
        for i in range(3):
            # Parse the function's arguments
            try:
                function_call_args = response.choices[0].message.function_call.arguments
                sentiment_analysis_result = json.loads(function_call_args)  # Safely parse arguments
                sentiment = sentiment_analysis_result['sentiment']  # Default to 0 if not provided
                predictions.append(sentiment)
                break

            except Exception as e:
                if i == 2:
                    print(sentiment_analysis_result)
                    print("Error parsing function call arguments:", e)
                    predictions.append(0)

    return predictions


In [31]:
# article = "The stock market is crashing and the economy is in shambles. The unemployment rate is at an all-time high and people are struggling to make ends meet. The government is doing nothing to help and the future looks bleak."
# eval(article)

In [3]:
from datasets import load_dataset

dataset = load_dataset("fhamborg/news_sentiment_newsmtsc", trust_remote_code=True)

In [5]:
test_data = dataset['test']

In [25]:
test_data[0]

{'mention': 'Seth',
 'polarity': 0,
 'from': 39,
 'to': 43,
 'sentence': 'Though we do not know what other items Seth may have had in his possession, his watch, phone, wallet and necklace were not stolen.',
 'id': 'polusa_v1_4307505_-1_11_Seth_39_43'}

In [35]:
results = predict_sentiment_logits(model, tokenizer, test_data)

In [45]:
results = predict_sentiment_with_function_calling(openai_client, test_data)

{}
Error parsing function call arguments: 'sentiment'


In [46]:
actual = test_data['polarity']

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(actual, results)

In [47]:
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6725
